#### For Colab

In [1]:
"""
function ClickConnect(){
    console.log("Working");
    document.querySelector("colab-toolbar-button").click() 
}
var i = setInterval(ClickConnect, 900000)
clearInterval(i)
"""

'\nfunction ClickConnect(){\n    console.log("Working");\n    document.querySelector("colab-toolbar-button").click() \n}\nvar i = setInterval(ClickConnect, 900000)\nclearInterval(i)\n'

In [2]:
!nvidia-smi

Mon Feb  1 14:49:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
drive_dir = '/content/drive/My Drive/MAGMA: Summarization/'

#### Install Libraries

In [5]:
%%capture

requirements_dir = (drive_dir + 'transformers/examples/seq2seq/').replace(' ', '\ ')
requirements_file = requirements_dir + 'requirements.txt'
!cd $requirements_dir; pip install -r $requirements_file

!pip install transformers==4.1.1
!pip install -U pyarrow
!pip install -U wandb
!pip install -U sentence-transformers
!pip install -U optuna
!pip install -U "ray[tune]"

## **Fine-tuning**

In [6]:
finetune_script = '"'+drive_dir+'transformers/examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+drive_dir+'transformers/examples/seq2seq/run_eval.py"'

### **Config**

In [7]:
import sys
sys.path.insert(0, drive_dir)
import config

import torch
torch.manual_seed = config.SEED

import wandb
wandb.login()
project_name = 'hp_search_para_wordembed'
%env WANDB_PROJECT=$project_name

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=hp_search_para_wordembed


### HP search

In [8]:
model_name_or_path = 'sshleifer/distilbart-cnn-12-6'

In [9]:
data_dir = '"'+drive_dir+'datasets/xsum"'

output_dir = '"'+drive_dir+'fine-tuning/hp_search_test"'

log_dir = drive_dir + '/logs'

In [10]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained('facebook/bart-large')
model_config.activation_dropout = 0
model_config.attention_dropout = 0
model_config.classif_dropout = 0
model_config.min_length = 11
model_config.max_length = 63
model_config.no_repeat_ngram_size = 3
model_config.num_beams = 2
model_config.task_specific_params['summarization']['length_penalty'] = 1
model_config.task_specific_params['summarization']['min_length'] = 11
model_config.task_specific_params['summarization']['max_length'] = 62
model_config.task_specific_params['summarization']['num_beams'] = 2

model_config_dir = '"'+drive_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

In [11]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_name_or_path \
--tokenizer_name $model_name_or_path \
--data_dir $data_dir \
--hyperparameters_search \
--fp16 \
--freeze_embeds --freeze_encoder \
--sortish_sampler \
--task summarization \
--max_source_length 256 \
--max_target_length 60 \
--val_max_target_length 60 \
--num_train_epochs 1 \
--n_train 50 \
--n_val 5 \
--logging_steps 1 --logging_first_step \
--per_device_train_batch_size 1 --per_device_eval_batch_size 1 \
--evaluation_strategy steps --eval_steps 3 \
--predict_with_generate \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

2021-02-01 14:51:58.937197: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
100% 306M/306M [00:12<00:00, 25.3MB/s]
02/01/2021 14:52:22 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True
02/01/2021 14:52:22 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/content/drive/My Drive/MAGMA: Summarization/fine-tuning/hp_search_test', overwrite_output_dir=True, do_train=False, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epo